In [1]:
import pandas as pd

In [8]:
df = pd.read_csv('tweets_translated.csv')
df = df.drop(df.columns[0], axis=1)

In [9]:
df.isnull().sum(axis=0)

id                     0
conversation_id        0
created_at             0
date                   0
time                   0
timezone               0
user_id                0
username               0
name                   1
place              73272
tweet                  0
language               0
mentions               0
urls                   0
photos                 0
replies_count          0
retweets_count         0
likes_count            0
hashtags               0
cashtags               0
link                   0
retweet                0
quote_url          71042
video                  0
thumbnail          60363
near               73305
geo                73305
source             73305
user_rt_id         73305
user_rt            73305
retweet_id         73305
reply_to               0
retweet_date       73305
translate          73305
trans_src          73305
trans_dest         73305
time_stamp             0
Unnamed: 0.3        5532
Unnamed: 0.2       16739
Unnamed: 0.1       21194


In [10]:
#Droping all coulumns that are empty or with many missing values.
df = df.drop(['retweet_date', 'translate', 'trans_src', 'trans_dest', 'retweet_id', 
             'user_rt', 'user_rt_id', 'geo', 'near', 'place', 'source', 'quote_url', 'timezone', 'conversation_id', 'created_at'], axis=1)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73305 entries, 0 to 73304
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              73305 non-null  int64 
 1   date            73305 non-null  object
 2   time            73305 non-null  object
 3   user_id         73305 non-null  int64 
 4   username        73305 non-null  object
 5   name            73304 non-null  object
 6   tweet           73305 non-null  object
 7   language        73305 non-null  object
 8   mentions        73305 non-null  object
 9   urls            73305 non-null  object
 10  photos          73305 non-null  object
 11  replies_count   73305 non-null  int64 
 12  retweets_count  73305 non-null  int64 
 13  likes_count     73305 non-null  int64 
 14  hashtags        73305 non-null  object
 15  cashtags        73305 non-null  object
 16  link            73305 non-null  object
 17  retweet         73305 non-null  bool  
 18  video 

In [15]:
df = df.drop_duplicates(subset='id')

In [16]:
df.shape

(71875, 22)

In [17]:
len(df['id'].unique())

71875

## Cleaning Tweets 

In [18]:
def hashtag_removal(r):
    """
    function to remove hashtags from a tweet
    """
    twt=r['tweet'].lower()
    tags=list(r['hashtags'][1:-1].split(','))
    for i in tags:
        i=i.strip()
        i=i[1:-1]
        twt=twt.replace('#'+i,'')
    return twt 
df['clean_tweet']=df.apply(lambda x:hashtag_removal(x),axis=1)

In [19]:
!pip install tweet-preprocessor
import preprocessor as p
#Clening the tweets using tweet-preprocesor
df['clean_tweet']=df['clean_tweet'].apply(lambda x:p.clean(x))

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [20]:
import regex as re
def punctuation_removal(r):
    """
    Removing punctioion from tweet
    """
    ls=re.findall(r'&(\w+);', r)
    for i in ls:
        r=r.replace(f'&{i};','')
    punc = '’’!()-[]{};:’”\,<>./?@#$%^&*_~’’' 
    for ele in r:
         if ele in punc:
            r = r.replace(ele, '')
         return r
df['clean_tweet']=df['clean_tweet'].apply(lambda x:punctuation_removal(x))

In [21]:
#Remove Digits
df['clean_tweet'] = df['clean_tweet'].astype(str).str.replace(' \d+', '')
df['clean_tweet']

0            ridiculous .... that's how it goes in formula
1        as a fan for years i never agreed with the sta...
2        it's written in my will to have the same, i'm ...
3        the hell of formula workers in montmel: no wat...
4                    i kno sorry formula fan ain't talkin'
                               ...                        
73300             tesla should compete in formula electric
73301            what every ferrari f1 driver is scared of
73302      grande charles, grande carlos, really good work
73303    pierre said the only french food yuki hasnt tr...
73304    you're not going to argue because there's no f...
Name: clean_tweet, Length: 71875, dtype: object

## Tokenization

In [22]:
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.tokenize import TweetTokenizer
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
def lemmatize_text(text):
 return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
df['tokenized_tweet'] = df['clean_tweet'].apply(lambda x:lemmatize_text(x))
df['tokenized_tweet']

[nltk_data] Downloading package wordnet to /home/repl/nltk_data...
[nltk_data] Downloading package stopwords to /home/repl/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /home/repl/nltk_data...


0        [ridiculous, ..., that's, how, it, go, in, for...
1        [a, a, fan, for, year, i, never, agreed, with,...
2        [it's, written, in, my, will, to, have, the, s...
3        [the, hell, of, formula, worker, in, montmel, ...
4          [i, kno, sorry, formula, fan, ain't, talkin, ']
                               ...                        
73300      [tesla, should, compete, in, formula, electric]
73301    [what, every, ferrari, f1, driver, is, scared,...
73302    [grande, charles, ,, grande, carlos, ,, really...
73303    [pierre, said, the, only, french, food, yuki, ...
73304    [you're, not, going, to, argue, because, there...
Name: tokenized_tweet, Length: 71875, dtype: object

**Stop Words removal**

In [23]:
stop_words = set(stopwords.words('english'))
df['tokenized_tweet'] = df['tokenized_tweet'].apply(lambda x: [item for item in x if item not in stop_words])

In [24]:
df = df.sort_values('date')

In [25]:
df.to_csv('cleand_tweets.csv', index=False)